## Chatflow user interface

In [2]:
import requests
import json
from typing import Dict, Any
from http import HTTPStatus

class WorkflowAPIError(Exception):
    """Custom exception for API-related errors"""
    def __init__(self, message: str, status_code: int = None, response_body: str = None):
        super().__init__(message)
        self.status_code = status_code
        self.response_body = response_body

def parse_event_stream(response_text: str) -> Dict[str, Any]:
    """
    Parse the streamed response and extract the final meaningful event.
    """
    events = [line.strip() for line in response_text.split('\n') if line.strip().startswith('data: ')]
    
    for event in reversed(events):
        try:
            data = json.loads(event[6:])  # remove 'data: '
            if data.get("event") == "message":
                return {
                    "message_id": data.get("message_id"),
                    "conversation_id": data.get("conversation_id"),
                    "answer": data.get("answer"),
                    "metadata": data.get("metadata", {})
                }
        except json.JSONDecodeError:
            continue

    return {}

def run_workflow(api_key: str, query: str, base_url: str = "http://localhost") -> Dict[str, Any]:
    """
    Run a workflow via API and return the outputs.
    
    Args:
        api_key (str): API key for authentication
        query (str): The query/issue to process
        base_url (str): Base URL for the API endpoint
        
    Returns:
        Dict[str, Any]: The outputs from the workflow response
        
    Raises:
        WorkflowAPIError: If the API request fails or response is invalid
    """
    endpoint = f"{base_url}/v1/chat-messages"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    
    # Simplified payload structure with issue parameter
    payload = {
        "inputs": {
            "code_snippet": query
        },
        "query": "What is the issue with this code snippet?",
        "response_mode": "streaming",
        "conversation_id": "",
        "user": "456104a6-c8f9-4172-b5c5-ad07ecc2573b"
    }
    
    try:
        # Print request details for debugging
        # print(f"Making request to: {endpoint}")
        # print(f"Headers: {json.dumps(headers, indent=2)}")
        # print(f"Payload: {json.dumps(payload, indent=2)}")
        
        response = requests.post(endpoint, headers=headers, json=payload, stream=True)
        
        # Print response details for debugging
        # print(f"\nResponse Status: {response.status_code}")
        # print(f"Response Headers: {dict(response.headers)}")
        
        # For streaming responses, collect the entire response
        response_text = ''
        for chunk in response.iter_content(decode_unicode=True):
            if chunk:
                response_text += chunk.decode('utf-8') if isinstance(chunk, bytes) else chunk
                # print(f"Response Body: {chunk.decode('utf-8') if isinstance(chunk, bytes) else chunk}")
        
        if response.status_code != 200:
            raise WorkflowAPIError(
                "Invalid request. Please check the request parameters.",
                response.status_code,
                response_text
            )
        
        # Parse the event stream response
        outputs = parse_event_stream(response_text)
        if outputs:
            return outputs
        else:
            raise WorkflowAPIError(
                "No outputs found in the event stream response",
                response.status_code,
                response_text
            )
            
    except requests.exceptions.RequestException as e:
        if hasattr(e, 'response') and e.response is not None:
            raise WorkflowAPIError(
                f"API request failed: {str(e)}",
                e.response.status_code,
                e.response.text if hasattr(e.response, 'text') else None
            )
        raise WorkflowAPIError(f"API request failed: {str(e)}")
    except json.JSONDecodeError as e:
        raise WorkflowAPIError(f"Failed to parse response JSON: {str(e)}")

def ape_audit(query: str) -> Dict[str, Any]:
    """
    Executes the API call with the given query and returns results.
    """
    api_key = "app-3aKWXydnT5hy9Ois9JfEvyMi"
    
    try:
        outputs = run_workflow(api_key, query)
        return outputs
        # print("\nWorkflow outputs:", outputs)
    except WorkflowAPIError as e:
        print(f"Error: {str(e)}")
        if e.status_code:
            print(f"Status Code: {e.status_code}")
        if e.response_body:
            print(f"Response Body: {e.response_body}")

### Sample Test

In [4]:
sample_query = '''pragma solidity ^0.4.25;

contract TimedCrowdsale {
  
  function isSaleFinished() view public returns (bool) {
    
    return block.timestamp >= 1546300800;
  }
}'''
result = ape_audit(sample_query)
if result is not None:
    # Process the outputs
    print(result)  # 

{'message_id': '8244a72f-2897-4558-a9a5-7353e8881425', 'conversation_id': 'b0d65c23-a9b9-45b4-a4c4-3cff8de14a80', 'answer': '{"summary": "The audit report for the `TimedCrowdsale` contract identifies several issues categorized by severity:", "findings": [{"issue": "Outdated Solidity Version", "severity": "High", "description": "The contract uses Solidity `^0.4.25`, which lacks critical security features like overflow/underflow checks.", "impact": "Increased risk of arithmetic vulnerabilities.", "recommendation": "Upgrade to Solidity `0.8.x` or later."}, {"issue": "Hardcoded Timestamp", "severity": "Medium", "description": "The deadline timestamp is hardcoded, limiting flexibility.", "impact": "Requires redeployment for changes, reducing usability.", "recommendation": "Make the deadline configurable via constructor or setter function."}, {"issue": "Potential Miner Manipulation", "severity": "Medium", "description": "Reliance on `block.timestamp` makes the contract susceptible to minor m

## Workflow API

In [10]:
import requests
import json
from typing import Dict, Any
from http import HTTPStatus

class WorkflowAPIError(Exception):
    """Custom exception for API-related errors"""
    def __init__(self, message: str, status_code: int = None, response_body: str = None):
        super().__init__(message)
        self.status_code = status_code
        self.response_body = response_body

def parse_event_stream(response_text: str) -> Dict[str, Any]:
    """
    Parse the event stream response and extract the final outputs.
    
    Args:
        response_text (str): The raw event stream response text
        
    Returns:
        Dict[str, Any]: The parsed outputs from the event stream
    """
    # Split the response into individual events
    events = [line.strip() for line in response_text.split('\n') if line.strip().startswith('data: ')]
    
    # Parse each event and return the last complete one
    for event in reversed(events):
        try:
            data = json.loads(event[6:])  # Remove 'data: ' prefix and parse JSON
            if data.get('event') == 'workflow_finished':
                workflow_data = data['data']
                return {
                    'outputs': workflow_data.get('outputs', {}),
                    'total_tokens': workflow_data.get('total_tokens'),
                    'total_steps': workflow_data.get('total_steps')
                }
        except json.JSONDecodeError:
            continue
    
    return {}

def run_workflow(api_key: str, query: str, base_url: str = "http://localhost") -> Dict[str, Any]:
    """
    Run a workflow via API and return the outputs.
    
    Args:
        api_key (str): API key for authentication
        query (str): The query/issue to process
        base_url (str): Base URL for the API endpoint
        
    Returns:
        Dict[str, Any]: The outputs from the workflow response
        
    Raises:
        WorkflowAPIError: If the API request fails or response is invalid
    """
    endpoint = f"{base_url}/v1/workflows/run"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    
    # Simplified payload structure with issue parameter
    payload = {
        "inputs": {
            "code_snippet": query
        },
        "response_mode": "streaming",
        "user": "f287e313-99d8-4e37-b9d8-18ba123c6efa"
    }
    
    try:
        
        response = requests.post(endpoint, headers=headers, json=payload, stream=True)
        
        response_text = ''
        for chunk in response.iter_content(decode_unicode=True):
            if chunk:
                response_text += chunk.decode('utf-8') if isinstance(chunk, bytes) else chunk
                # print(f"Response Body: {chunk.decode('utf-8') if isinstance(chunk, bytes) else chunk}")
        
        if response.status_code != 200:
            raise WorkflowAPIError(
                "Invalid request. Please check the request parameters.",
                response.status_code,
                response_text
            )
        
        # Parse the event stream response
        outputs = parse_event_stream(response_text)
        if outputs:
            return outputs
        else:
            raise WorkflowAPIError(
                "No outputs found in the event stream response",
                response.status_code,
                response_text
            )
            
    except requests.exceptions.RequestException as e:
        if hasattr(e, 'response') and e.response is not None:
            raise WorkflowAPIError(
                f"API request failed: {str(e)}",
                e.response.status_code,
                e.response.text if hasattr(e.response, 'text') else None
            )
        raise WorkflowAPIError(f"API request failed: {str(e)}")
    except json.JSONDecodeError as e:
        raise WorkflowAPIError(f"Failed to parse response JSON: {str(e)}")

def ape_audit(query: str) -> Dict[str, Any]:
    """
    Main function to run the workflow and print outputs.
    
    Args:
        query (str): The query/issue to process
    """
    api_key = "app-SDOVFK5BswWDy0XFEbJsiHDB"
    
    try:
        outputs = run_workflow(api_key, query)
        return outputs
        # print("\nWorkflow outputs:", outputs)
    except WorkflowAPIError as e:
        print(f"Error: {str(e)}")
        if e.status_code:
            print(f"Status Code: {e.status_code}")
        if e.response_body:
            print(f"Response Body: {e.response_body}")

In [8]:
sample_query = '''pragma solidity ^0.4.25;

contract TimedCrowdsale {
  
  function isSaleFinished() view public returns (bool) {
    
    return block.timestamp >= 1546300800;
  }
}'''
result = ape_audit(sample_query)
if result is not None:
    # Process the outputs
    print(result)  # 

{'outputs': {'final_findings': '{"summary": "The audit report identifies several issues in the TimedCrowdsale smart contract, categorized by severity and impact:", "findings": [{"issue": "Hardcoded Timestamp", "severity": "Medium", "impact": "Lack of flexibility requiring redeployment for deadline changes.", "exploit": "None directly, but inflexibility may lead to operational inefficiencies."}, {"issue": "Block Timestamp Manipulation Risk", "severity": "Low", "impact": "Miners can slightly manipulate timestamps (~15 seconds).", "exploit": "Unlikely to be significant but could theoretically affect sale timing."}, {"issue": "Lack of Events", "severity": "Low", "impact": "Reduced transparency and off-chain tracking capabilities.", "exploit": "None, but hinders monitoring and auditing."}, {"issue": "No Owner Controls", "severity": "Medium", "impact": "No mechanism to handle incorrect timestamps or emergencies.", "exploit": "Potential for irreversible issues if the deadline is misconfigured

In [11]:
import pandas as pd
import json


def save_to_csv(code_snippet, manager_findings, auditor_findings, programmer_findings, final_report, file_path, execution_time, total_tokens, total_steps):
    """
    Save code_snippet, manager_findings, auditor_findings, and final_report to the same row as the existing code_snippet.

    If the code_snippet already exists in the file, update its row.
    If it doesn't exist, append it as a new row.

    Parameters:
        code_snippet (str): The code snippet string.
        manager_findings (str): The audit plan string.
        auditor_findings (str): The initial analysis string.
        final_report (str): The final report string.
        file_path (str): The path to the CSV file.
    """
    try:
        # Create a DataFrame with the new data
        new_data = {
            "code_snippet": code_snippet,
            "manager_findings": manager_findings,
            "auditor_findings": auditor_findings,
            "programmer_findings": programmer_findings,
            "final_findings": final_report,
            "execution_time": execution_time,
            "total_tokens": total_tokens,
            "total_steps": total_steps
        }

        try:
            # Load existing data if the file exists
            existing_data = pd.read_csv(file_path)

            # Check if the code_snippet exists in the file
            if code_snippet in existing_data["code_snippet"].values:
                # Update the existing row
                existing_data.loc[existing_data["code_snippet"] == code_snippet, [
                    "manager_findings", "auditor_findings", "programmer_findings", "final_findings", "execution_time", "total_tokens", "total_steps"
                ]] = manager_findings, auditor_findings, programmer_findings, final_report, execution_time, total_tokens, total_steps
            else:
                # Append the new row if code_snippet does not exist
                existing_data = pd.concat([
                    existing_data,
                    pd.DataFrame([new_data])
                ], ignore_index=True)

            # Save the updated data back to the file
            existing_data.to_csv(file_path, index=False)
        except FileNotFoundError:
            # If the file does not exist, create it with the new data
            pd.DataFrame([new_data]).to_csv(file_path, index=False)

        print(f"Data successfully saved to {file_path}")
    except Exception as e:
        print(f"Error saving data to CSV: {str(e)}")


In [12]:
from typing import Dict, Any, Tuple
import re
import json
import pandas as pd
import time
            
def extract_findings(json_str: str):
    # Parse the outer JSON structure
    try:
        outer_data = json.loads(json_str)
        if outer_data is None:
            print("Warning: json.loads returned None")
            return None, None, None
    except json.JSONDecodeError as e:
        print(f"Error parsing outer JSON structure: {e}")
        return None, None, None
    
    def clean_json_field(field_str):
        """
        Removes Markdown code fences from a JSON string.
        """
        # Use regular expressions to extract content between ```json and ```
        pattern = r"```json\s*\n(.*?)\n```"
        match = re.search(pattern, field_str, re.DOTALL)
        if match:
            return match.group(1)
        else:
            # If no code fences are found, return the original string
            return field_str
    
    # Clean each inner JSON string
    final_report_str = clean_json_field(outer_data.get('final_findings', ''))
    manager_findings_str = clean_json_field(outer_data.get('manager_findings', ''))
    auditor_findings_str = clean_json_field(outer_data.get('auditor_findings', ''))
    programmer_findings_str = clean_json_field(outer_data.get('programmer_findings', ''))
    
    # Parse the cleaned JSON strings
    try:
        final_report = json.loads(final_report_str)
    except json.JSONDecodeError as e:
        print("Error decoding 'final report':", e)
        final_report = None
    
    try:
        manager_findings = json.loads(manager_findings_str)
    except json.JSONDecodeError as e:
        print("Error decoding 'audit plan':", e)
        manager_findings = None
    
    try:
        auditor_findings = json.loads(auditor_findings_str)
    except json.JSONDecodeError as e:
        print("Error decoding 'initial analysis':", e)
        auditor_findings = None
    
    try:
        programmer_findings = json.loads(programmer_findings_str)
    except json.JSONDecodeError as e:
        print("Error decoding 'initial analysis':", e)
        programmer_findings = None
    
    return final_report, manager_findings, auditor_findings, programmer_findings

def clean_markdown_json_block(text: str) -> str:
    """
    Remove markdown fences like ```json ... ``` and trim whitespace.
    """
    # Remove triple backticks and optional 'json' specifier
    cleaned = re.sub(r"^```(?:json)?\s*|\s*```$", "", text.strip(), flags=re.IGNORECASE | re.MULTILINE)
    return cleaned.strip()
    

In [16]:
if __name__ == "__main__":
    # with open('paste.txt', 'r') as file:
    #     json_str = file.read()
    #     print(json_str)
    
    file_path = '/Users/weizhiyuan/Documents/paper/3.LLMSmartAudit/contracts/code4rena-llmsmartaudit-deepseek/3_processed_2021-04-marginswap.csv'

    print(f"\nFile path: {file_path}")
    
    execution_times = []

    # Load the CSV file
    data = pd.read_csv(file_path)
    start_index = 0
    
    for index, row in data.iloc[start_index:].iterrows():
        start_time = time.time()  # Start timing
        contract = row['code_snippet']
        
        if pd.isna(contract) or str(contract).strip() == "":
            print(f"Empty contract found at index {index}. Stopping execution.")
            execution_times.append(None)
            break
    
        # api_key = "app-c48Ngaev7VcPm9TLDQacVcZP"
    
        parsed_data = ape_audit(contract)
        
        try:
    
            try:
                
                # Step 1: Extract 'text' from parsed_data
                raw_text = parsed_data.get('outputs', {})
                
                print("raw_text", raw_text)

                # Step 2: If it's a string, try to parse it as JSON
                if isinstance(raw_text, str):
                    try:
                        text_fields = json.loads(raw_text)
                    except json.JSONDecodeError:
                        text_fields = {}
                elif isinstance(raw_text, dict):
                    text_fields = raw_text
                else:
                    text_fields = {}
                    
                # Step 3: Extract fields
                final_report = clean_markdown_json_block(text_fields.get("final_findings", ""))
                manager_findings = clean_markdown_json_block(text_fields.get("manager_findings", ""))
                auditor_findings = clean_markdown_json_block(text_fields.get("auditor_findings", ""))
                programmer_findings = clean_markdown_json_block(text_fields.get("programmer_findings", ""))
                
                # Extract token and step metrics
                total_tokens = parsed_data.get('total_tokens', None)
                total_steps = parsed_data.get('total_steps', None)
                
                # Calculate execution time
                end_time = time.time()
                execution_time = end_time - start_time
                execution_times.append(execution_time)
                
                # Save to CSV
                output_file = file_path
                save_to_csv(contract, manager_findings, auditor_findings, programmer_findings, final_report, file_path, execution_time, total_tokens, total_steps)
                # print(f"Successfully saved data to {file_path}")
                print(index)
            except ValueError as e:
                print(f"Error: {str(e)}")

        except (TypeError, ValueError) as e:
            print(f"Error converting outputs to JSON: {e}")
            continue

        # if (index + 1) % 2 == 0:
            print(f"Completed {index + 1} contracts. Sleeping for 5 minutes.")
        time.sleep(300)  # Sleep for 5 minutes
            
print(f"Successfully saved data to {file_path}")


File path: /Users/weizhiyuan/Documents/paper/3.LLMSmartAudit/contracts/code4rena-llmsmartaudit-deepseek/3_processed_2021-04-marginswap.csv
raw_text {'final_findings': '{"summary": "The audit report for the MarginRouter contract identifies several bugs categorized by severity. Below is a concise breakdown of each issue:", "issues": [{"severity": "High", "description": "Reentrancy Risk in External Calls", "impact": "Functions like `crossDeposit`, `crossWithdraw`, and `crossBorrow` are vulnerable to reentrancy attacks due to external calls before state updates.", "recommendation": "Follow the Checks-Effects-Interactions (CEI) pattern by updating state before making external calls."}, {"severity": "Medium", "description": "Insufficient Access Control", "impact": "Functions such as `authorizedSwapExactT4T` and `authorizedSwapT4ExactT` lack role definitions, risking unauthorized access.", "recommendation": "Implement role-based access control using modifiers like `onlyRole`."}, {"severity":